In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'C:\Users\nikod\Documents\PythonProjects\hivPKDproject\paper':
  os.chdir(r'C:\Users\nikod\Documents\PythonProjects\hivPKDproject\paper')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\nikod\AppData\Local\Temp\ipykernel_14912\3534258046.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\importlib\\_bootstrap.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\importlib\\_bootstrap_external.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\zipimport.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\codecs.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\encodings\\aliases.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\encodings\\__init__.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\encodings\\utf_8.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\encodings\\cp1250.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\abc.py": 1696245958.0, "C:\\Users\\nikod\\AppData\\Local\\Programs\\Python\\Python

In [2]:
#| include: false

#execute: 
#  cache: true

# loading packages

import seaborn as sns
# import arviz as az           arviz error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import subprocess


# Complex data wrangling and model building is kept in seperate files,
# please read the readme file to navigate easily through the project


# color palletes:

pallette_colBlindPair1 = ["#DDCC77", "#88CCEE"] 
fancyMine1 = ["#024b7a", "#44b7c2"]


In [3]:
#| label: fig-demography
#| echo: false
#| fig-cap: 'The left plot shows the dataset''s gender and sexual identity distribution, while the right plot displays the ratio among those who tested positive.'
#| fig-pos: H
#| warning: false


# File containing the wrangling of a dataset: ../pythonCode/DataWranglinfForHomoCasualityCheck.py

PKD_model_DF = pd.read_csv("../dataBits/PKD_model_DF.csv")
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1st plot - Gender Distribution by Sexual Identity
grouped_data = PKD_model_DF.groupby(["Płeć", "Hetero_normative"]).size().unstack(fill_value=0)
grouped_data = grouped_data.loc[grouped_data.index != 'I']



ax1 = grouped_data.plot(kind="bar", stacked=True, color=fancyMine1, width=0.4,  ax=axes[0])
ax1.set_xlabel("")
ax1.set_title("Gender Distribution by Sexual Identity", fontsize=24, y=1.1)
ax1.legend(labels=["Non-Hetero norm.", "Hetero norm."], frameon=False, fontsize= 18, loc='upper left', bbox_to_anchor=(0, 1))
ax1.set_xticklabels(['Females', 'Males'], rotation=0)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.grid(False)

ax1.tick_params(axis='y', labelsize=18)
ax1.tick_params(axis='x', labelsize=18)

# 2nd plot - HIV Positive Ratio
category_counts = PKD_model_DF.groupby(["Płeć", "Hetero_normative", "HIV"]).size().reset_index(name='Count')
category_counts = category_counts.loc[category_counts['Płeć'] != 'I']
category_counts = category_counts.loc[category_counts['HIV'] != 0]
category_counts['Percentage'] = category_counts['Count'] / sum(category_counts['Count'])

ax2 = sns.barplot(data=category_counts, x='Płeć', y='Percentage',
                   hue='Hetero_normative', palette=fancyMine1, saturation=1,  ax=axes[1])
ax2.set_xlabel('')
ax2.set_ylabel('')
ax2.set_title('HIV Positive', fontsize=22, y=1.1)
ax2.set_xticklabels(['Females', 'Males'], rotation=0)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.legend().set_visible(False)

ax2.tick_params(axis='y', labelsize=18)
ax2.tick_params(axis='x', labelsize=18)

ax2.grid(False)


from matplotlib.ticker import FuncFormatter
def percentage_formatter(x, pos):
    return f'{x:.0%}'  # Display percentage without decimal places

ax2.yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

ax2.set_yticks([0.0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70])



plt.tight_layout()
plt.show()


C:\Users\nikod\AppData\Local\Temp\ipykernel_14912\2196785879.py:35: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.



<Figure size 3600x1500 with 2 Axes>

In [4]:
#| eval: false

dat_listGender = {
    'HIV': PKD_model_DF.HIV.values,
    'Hetero_normative': PKD_model_DF.Hetero_normative.values,
    'Gender_encoded': PKD_model_DF.Gender_encoded.values,
    }

def model_hetero_Gender(Hetero_normative, Gender_encoded, HIV=None):     
    a = numpyro.sample("a", dist.Normal(0, 1.5))  
    b = numpyro.sample("b", dist.Normal(0, 0.5).expand([2])) 
    c = numpyro.sample("c", dist.Normal(0, 0.5).expand([2])) 

    logit_p = a + b[Hetero_normative] + c[Gender_encoded]
    numpyro.sample("HIV", dist.Binomial(logits=logit_p), obs=HIV)

HIV_HetreoNorm_Gender = MCMC(NUTS(model_hetero_Gender),
    num_warmup=500, num_samples=500) 
HIV_HetreoNorm_Gender.run(random.PRNGKey(0), **dat_listGender)


In [5]:
#| label: fig-modelsexhetero
#| fig-cap: This visualization displays the predictions generated by the logistic regression model trained on the dataset. The violins depict the probability distribution of HIV acquisition predictions with respect to gender and sexual identity.
#| echo: false
#| warning: false
#| fig-pos: H

# File containing the model training: ../pythonCode/MLogit_Partners_Heteronorm_gender.py

with open('../savedBits/postDF_Hetero_Gender.pkl', 'rb') as f:
    postDF_Hetero_Gender = pickle.load(f)


gender_mapping = {0: "Females", 1: "Males"}

postDF_Hetero_Gender['Gender'] = postDF_Hetero_Gender['Gender'].replace(gender_mapping)

sex_ident_order = ['Hetero', 'Non-hetero']


postDF_Hetero_Gender['SexIdent'] = pd.Categorical(postDF_Hetero_Gender['SexIdent'], categories=sex_ident_order, ordered=True)

sns.set(style="whitegrid", font_scale=0.85)  # Adjust font_scale as needed (0.9 makes it slightly smaller)
g = sns.FacetGrid(postDF_Hetero_Gender, col='Gender' ) 
g.map_dataframe(sns.violinplot, x="SexIdent", hue='SexIdent', y="Probability",
palette= fancyMine1)# pallete here

g.set_axis_labels('', 'Probability')
g.set_titles(col_template = '{col_name}')

g.fig.subplots_adjust(top=0.77)
g.fig.suptitle('Probability of HIV Infection')

plt.show()

<Figure size 1800x900 with 2 Axes>

In [6]:
#| label: fig-dagwrong
#| fig-cap: 'DAG representing the naive relation between variables resembling the first model. ''U'' stands for unobserved, ~H for non-hetero, while the rest of the labels are self-explanatory.'
#| echo: false
#| warning: false
#| fig-align: center




dagWrong = nx.DiGraph()
dagWrong.add_edges_from([('Male', 'HIV'), ('~H', 'HIV'), ('U', 'HIV')])

# Define node colors
node_colors = {
    'Male': '#D1E5F4',  
    '~H': '#D1E5F4',  
    'U': '#AEAEAE',        
    'HIV': '#D1E5F4'    
}

plt.figure(figsize=(3.5, 2.7))

pos = nx.spring_layout(dagWrong, seed=42)
nx.draw(dagWrong, pos, with_labels=True, node_color=[node_colors[node] for node in dagWrong.nodes()], node_size=1000)


plt.show()


<Figure size 1050x810 with 1 Axes>

In [7]:
#| echo: false
#| fig-align: center
#| label: fig-violinbigmales
#| fig-cap: 'In this visualization, you can observe various categories considered as contributors to a risky profile. This plot focuses solely on males.'

# File containing the model training: ../pythonCode/MLog_homRIskProfiler.py

DFPost_HIVRiskProfile = pd.read_csv("../savedBits/DFPost_HIVRiskProfile.csv")

sns.set(style="whitegrid")

custom_palette = ["#8D4585", "#56B4E9", '#FFC20A'] 

from matplotlib.lines import Line2D

postDF_RiskProfileMales = DFPost_HIVRiskProfile[DFPost_HIVRiskProfile['Gender'] == 1]
postDF_RiskProfileFemales = DFPost_HIVRiskProfile[DFPost_HIVRiskProfile['Gender'] == 0]

postDF_Alco1 = postDF_RiskProfileMales[postDF_RiskProfileMales['Sex_alcohol'] == 1]
postDF_Alco0 = postDF_RiskProfileMales[postDF_RiskProfileMales['Sex_alcohol'] == 0]

fig, axes = plt.subplots(2, 1, figsize=(14.5, 8.5))  

sns.violinplot(data=postDF_Alco1, x="AnalPosition", y="Probability", hue="PartnersNum", order= ['no', 'active', 'passive', 'vers'], ax=axes[0], palette=custom_palette)
axes[0].set_title("Sex After Alcohol", fontsize=20)
axes[0].set_xlabel("Anal Sex Preference", fontsize=20)
axes[0].set_ylabel("Probability", fontsize=20)

axes[0].tick_params(axis='y', labelsize=18)
axes[0].tick_params(axis='x', labelsize=18)

legend_handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[0], markersize=10, label='1-10'),
                  Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[1], markersize=10, label='11-50'), # '1-10', '11-50', 'above_51'
                  Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[2], markersize=10, label='above 51')]


legend1 = axes[0].legend(title="Number of Partners", loc="upper left", handles=legend_handles, fontsize=18)
legend1.get_title().set_fontsize('18')
legend1.get_frame().set_alpha(None)

# Plot for gender = 0
sns.violinplot(data=postDF_Alco0, x="AnalPosition", y="Probability", hue="PartnersNum", order= ['no', 'active', 'passive', 'vers'], ax=axes[1], palette=custom_palette)
axes[1].set_title("No Sex After Alcohol", fontsize=20)
axes[1].set_xlabel("Anal Sex Preference", fontsize=20)
axes[1].set_ylabel("Probability", fontsize=20)

axes[1].tick_params(axis='y', labelsize=18)
axes[1].tick_params(axis='x', labelsize=18)


legend2 = axes[1].legend(title="Number of Partners", loc="upper left", handles=legend_handles, fontsize=18)
legend2.get_title().set_fontsize('18')
legend2.get_frame().set_alpha(None)

# unifying y axis values
combined_data = DFPost_HIVRiskProfile["Probability"]
y_min = combined_data.min()
y_max = combined_data.max()
axes[0].set_ylim(y_min, y_max)
axes[1].set_ylim(y_min, y_max)

plt.suptitle("Probability of Getting HIV Infected (Males Only)", fontsize=22)

plt.tight_layout()
plt.show()

<Figure size 4350x2550 with 2 Axes>

In [8]:
#| echo: false
#| fig-align: center
#| label: fig-violinbigfemales
#| fig-cap: 'This is a twin visualization covering risky categories. This plot focuses exclusively on females, and the difference in the probability of getting infected is striking.'


# File containing the model training: ../pythonCode/MLog_homRIskProfiler.py

sns.set(style="whitegrid")

custom_palette = ["#8D4585", "#56B4E9", '#FFC20A'] 


postDF_Alco1 = postDF_RiskProfileFemales[postDF_RiskProfileFemales['Sex_alcohol'] == 1]
postDF_Alco0 = postDF_RiskProfileFemales[postDF_RiskProfileFemales['Sex_alcohol'] == 0]

fig, axes = plt.subplots(2, 1, figsize=(14.5, 8.5))  

sns.violinplot(data=postDF_Alco1, x="AnalPosition", y="Probability", hue="PartnersNum", order= ['no', 'active', 'passive', 'vers'], ax=axes[0], palette=custom_palette)
axes[0].set_title("Sex After Alcohol", fontsize=20)
axes[0].set_xlabel("Anal Sex Preference", fontsize=20)
axes[0].set_ylabel("Probability", fontsize=20)
axes[0].tick_params(axis='y', labelsize=18)
axes[0].tick_params(axis='x', labelsize=18)

legend_handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[0], markersize=10, label='1-10'),
                  Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[1], markersize=10, label='11-50'), # '1-10', '11-50', 'above_51'
                  Line2D([0], [0], marker='o', color='w', markerfacecolor=custom_palette[2], markersize=10, label='above 51')]


legend1 = axes[0].legend(title="Number of Partners", loc="upper left", handles=legend_handles, fontsize=20)
legend1.get_title().set_fontsize('18')
legend1.get_frame().set_alpha(None)

# Plot for gender = 0
sns.violinplot(data=postDF_Alco0, x="AnalPosition", y="Probability", hue="PartnersNum", order= ['no', 'active', 'passive', 'vers'], ax=axes[1], palette=custom_palette)
axes[1].set_title("No Sex After Alcohol", fontsize=20)
axes[1].set_xlabel("Anal Sex Preference", fontsize=20)
axes[1].set_ylabel("Probability", fontsize=20)
axes[1].tick_params(axis='y', labelsize=18)
axes[1].tick_params(axis='x', labelsize=18)

legend2 = axes[1].legend(title="Number of Partners", loc="upper left", handles=legend_handles, fontsize=20)
legend2.get_title().set_fontsize('18')
legend2.get_frame().set_alpha(None)

# unifying y axis values
combined_data = DFPost_HIVRiskProfile["Probability"]
y_min = combined_data.min()
y_max = combined_data.max()
axes[0].set_ylim(y_min, y_max)
axes[1].set_ylim(y_min, y_max)

plt.suptitle("Probability of Getting HIV Infected (Females Only)", fontsize=22)

plt.tight_layout()
plt.show()

<Figure size 4350x2550 with 2 Axes>

In [9]:
#| label: fig-riskprofgenderhiv
#| fig-cap: 'The visualization represents the probability of getting infected for RP holders, divided into females and males.'
#| echo: false
#| warning: false


# File containing the model training: ../pythonCode/MLog_DataHIGHRISKProfile.py

with open('../savedBits/DFPOST_HIv_risk_gender.pkl', 'rb') as f:
    DFPOST_HIv_risk_gender = pickle.load(f)

sns.set(style="whitegrid", font_scale=0.85) 

gender_mapping = {0: "Females", 1: "Males"}
DFPOST_HIv_risk_gender['Gender'] = DFPOST_HIv_risk_gender['Gender'].replace(gender_mapping)

risk_mapping = {0: "Not risky", 1: "Risky"}
DFPOST_HIv_risk_gender['RiskFactor'] = DFPOST_HIv_risk_gender['RiskFactor'].replace(risk_mapping)


g = sns.FacetGrid(DFPOST_HIv_risk_gender, col='Gender')
g.map_dataframe(sns.violinplot, x="RiskFactor", hue="RiskFactor", y="Probability",
palette=fancyMine1)
plt.xlabel("")
plt.ylabel("Probability")

g.set_axis_labels('', 'Probability')
g.set_titles(col_template = '{col_name}')


g.fig.subplots_adjust(top=0.77)
g.fig.suptitle('Probability of HIV Infection')

plt.show()


<Figure size 1800x900 with 2 Axes>

In [10]:
#| echo: false
#| label: fig-twinriskprob
#| fig-cap: 'The visualization represents the probability of being in RP, with a division by gender on the left and by sexual identity on the right.'
#| warning: false

# File containing the model training: ../pythonCode/MLog_DataHIGHRISKProfile.py

with open('../savedBits/DFPost_Post_RISK_NormGendernoHetero.pkl', 'rb') as f:
    DFPost_Post_RISK_NormGendernoHetero = pickle.load(f)



fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True,)


gender_mapping = {"0": "Females", "1": "Males"}
DFPost_Post_RISK_NormGendernoHetero['Gender'] = DFPost_Post_RISK_NormGendernoHetero['Gender'].replace(gender_mapping)

sns.violinplot(data=DFPost_Post_RISK_NormGendernoHetero, x="Gender", y="Probability", ax=axs[0], palette= fancyMine1)
axs[0].set_xlabel("")
axs[0].tick_params(axis='y', labelsize=18)
axs[0].tick_params(axis='x', labelsize=18)

axs[0].set_ylabel("Probability", fontsize=18)


# second vis

with open('../savedBits/DFPost_POST_RISK_Hetero.pkl', 'rb') as f:
    DFPost_POST_RISK_Hetero = pickle.load(f)


hetero_mapping = {"0": "Non hetero", "1": "Hetero"}
DFPost_POST_RISK_Hetero['HeteroNorm'] = DFPost_POST_RISK_Hetero['HeteroNorm'].replace(hetero_mapping)


sns.violinplot(DFPost_POST_RISK_Hetero, x="HeteroNorm", y="Probability", ax=axs[1], order= ['Hetero', 'Non hetero'], palette= fancyMine1)
axs[1].set_xlabel("")
axs[1].set_ylabel("")
axs[1].tick_params(axis='y', labelsize=18)
axs[1].tick_params(axis='x', labelsize=18)


axs[0].set_ylim(0, 0.06)
axs[1].set_ylim(0, 0.06)

plt.tight_layout()

fig.subplots_adjust(top=0.83)
fig.suptitle("Probability of the Risk Profile", fontsize=22)

plt.show()


C:\Users\nikod\AppData\Local\Temp\ipykernel_14912\4219190447.py:14: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.


C:\Users\nikod\AppData\Local\Temp\ipykernel_14912\4219190447.py:32: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.




<Figure size 3600x1800 with 2 Axes>

In [11]:
#| label: fig-postdag
#| fig-cap: 'This DAG represents the relationship between demographic variables, the RP (risk profile), and HIV infection.'
#| echo: false



dag2 = nx.DiGraph()
dag2.add_edges_from([('Male', 'RP'), ('~H', 'RP'), ('U', 'RP'), ('RP', 'HIV')])

# Define node colors
node_colors = {
    'Male': '#D1E5F4',  
    '~H': '#D1E5F4',  
    'U': '#AEAEAE',        
    'RP': '#D1E5F4',  
    'HIV': '#D1E5F4' 
}


plt.figure(figsize=(3.5, 2.7))

pos = nx.spring_layout(dag2, seed=42)
nx.draw(dag2, pos, with_labels=True,
    node_color=[node_colors[node] for node in dag2.nodes()], node_size=1000)


plt.show()


<Figure size 1050x810 with 1 Axes>

In [12]:
#| echo: false
#| label: fig-agehist
#| fig-cap: 'f"The histogram of age is ({lower_bound}, {upper_bound})"'
#| warning: false

with open('../dataBits/ageDF.pkl', 'rb') as f:
    ageDF = pickle.load(f)

plt.hist(ageDF, bins=20, edgecolor='k', alpha=0.7)

plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution Histogram')

plt.show()

lower_bound = np.percentile(ageDF, 2.5)  
upper_bound = np.percentile(ageDF, 97.5) 

print(f"The 95% confidence interval is ({lower_bound}, {upper_bound})")


<Figure size 1650x1050 with 1 Axes>

The 95% confidence interval is (19.0, 55.0)
